In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [2]:
#raw = importData('E29_CHECKIN1_MACROANALYSIS_V1.csv').replace("No Response", np.nan)

In [3]:
def importData(csv):
    data = pd.read_csv(csv)
    return data

#Break down goal setting and team process into: goals, support, communication, work allocation, team role, and enjoyment
#Then, find the magnitude of such difficulties for each team
def getCategoryDifferences(df, teamNumberCol):
    #Subset df to just goal setting and team process quantitative questions
    columnsOfInterest = [teamNumberCol, "SharedGoalTeamAvg", "SharedGoalClassAvg", 
                         "IndvGoalTeamAvg", "IndvGoalClassAvg", "SupportTeamAvg", 
                         "SupportClassAvg", "CommunicateTeamAvg", "CommunicateClassAvg", 
                         "WorkAllocTeamAvg", "WorkAllocClassAvg", "RoleTeamAvg", 
                         "RoleClassAvg", "EnjoyTeamAvg", "EnjoyClassAvg", 
                        'ConflictResTeamAvg', 'ConflictResClassAvg', 'ConflictResClassStDev',
                        'EfficiencyTeamAvg', 'EfficiencyClassAvg', 'EfficiencyClassStDev',
                        'ExpressionTeamAvg', 'ExpressionClassAvg', 'ExpressionClassStDev']
    temp = df[columnsOfInterest]
    
    #Collapse rows so that 1 row = 1 team
    temp = temp.drop_duplicates()
    
    #Calculate all averages (= team's cumulative deviation from class average)
    temp['goalsDiff'] = (temp["SharedGoalTeamAvg"] + temp["IndvGoalTeamAvg"]) - (temp["SharedGoalClassAvg"] + temp["IndvGoalClassAvg"])
    temp['supportDiff'] = temp["SupportTeamAvg"] - temp["SupportClassAvg"]
    temp['communicateDiff'] = temp["CommunicateTeamAvg"] - temp["CommunicateClassAvg"]
    temp['workAllocationDiff'] = temp["WorkAllocTeamAvg"] - temp["WorkAllocClassAvg"]
    temp['teamRoleDiff'] = temp["RoleTeamAvg"] - temp["RoleClassAvg"]
    temp['enjoyDiff'] = temp["EnjoyTeamAvg"] - temp["EnjoyClassAvg"]
    temp['conflictResDiff'] = temp["ConflictResTeamAvg"] - temp["ConflictResClassAvg"]
    temp['efficiencyDiff'] = temp['EfficiencyTeamAvg'] - temp['EfficiencyClassAvg']
    temp['expressionDiff'] = temp['ExpressionTeamAvg'] - temp['ExpressionClassAvg']
    temp['overallDiff'] = round(temp['goalsDiff'] + temp['supportDiff'] + temp['communicateDiff'] + temp['workAllocationDiff'] + temp['teamRoleDiff'] + temp['enjoyDiff'] + temp['conflictResDiff'] + temp['expressionDiff'] + temp['efficiencyDiff'], 2)
    
    tempRanked = temp.sort_values(teamNumberCol, ascending=True).reset_index(drop=True)
    tempRanked = tempRanked[[teamNumberCol, 'overallDiff', 'goalsDiff', 'supportDiff', 
                             'communicateDiff', 'workAllocationDiff', 'teamRoleDiff', 'enjoyDiff',
                            'conflictResDiff', 'efficiencyDiff', 'expressionDiff']]
    
    return tempRanked

#Assign each team a percentile (score) based on their overall difference from the class average. A difference of 0 designates a 50 (median)
def getTeamPercentiles(df, teamNumberCol):

    arr = np.array(df['overallDiff'])
    df["percentile"] = [round(stats.percentileofscore(arr, x), 2) for x in arr]

    return df
#Assign a category according to each team's percentile
def categorizePercentiles(df, teamNumberCol):
    arr = np.array(df['percentile'])
    cats = []
    #print(arr)
    for score in arr:
        if score >= 75:
            cats.append("Exceptional")
        elif score >= 50 and score < 75:
            cats.append("Strong")
        elif score >= 20 and score < 50:
            cats.append("Standard")
        elif score >= 10 and score < 20:
            cats.append("Weak")
        elif score >= 0 and score < 10:
            cats.append("Needs Attention")
        else:
            cats.append("Team did not respond")
    #print(cats)
    df["percentileCategory"] = cats
    return df

In [4]:
E29_CHECKIN01 = pd.read_csv("E29_CHECKIN01_MACROANALYSIS_V1.csv")

In [5]:
diffs = getCategoryDifferences(E29_CHECKIN01, 'TeamNumber')

In [6]:
percentiles = getTeamPercentiles(diffs, 'TeamNumber')

In [7]:
scored = categorizePercentiles(percentiles, 'TeamNumber')

In [8]:
scored.head()

,TeamNumber,overallDiff,goalsDiff,supportDiff,communicateDiff,workAllocationDiff,teamRoleDiff,enjoyDiff,conflictResDiff,efficiencyDiff,expressionDiff,percentile,percentileCategory
0,1,3.1,0.4,0.3,0.6,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong
1,2,-1.6,-0.1,0.0,0.1,-0.4,-0.5,0.1,-0.2,-0.8,0.2,41.67,Standard
2,3,-4.4,-1.1,-0.2,-0.9,-0.5,-0.3,-0.5,-0.2,-0.5,-0.2,23.33,Standard
3,4,-1.6,0.2,0.1,-0.3,-0.7,-0.4,-0.1,0.0,-0.5,0.1,41.67,Standard
4,5,-1.6,-0.3,-0.2,-0.1,-0.3,0.2,-0.3,-0.2,-0.3,-0.1,41.67,Standard


In [14]:
roster = pd.read_csv("E29_CHECKIN01_ROSTER_V1.csv")
roster = roster[["Group", "TeamNumber"]].drop_duplicates().reset_index()
macro = pd.read_csv("E29_CHECKIN01_MACROANALYSIS_V1.csv")
scored = scored

In [15]:
scored

,TeamNumber,overallDiff,goalsDiff,supportDiff,communicateDiff,workAllocationDiff,teamRoleDiff,enjoyDiff,conflictResDiff,efficiencyDiff,expressionDiff,percentile,percentileCategory
0,1,3.1,0.4,0.3,0.6,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong
1,2,-1.6,-0.1,0.0,0.1,-0.4,-0.5,0.1,-0.2,-0.8,0.2,41.67,Standard
2,3,-4.4,-1.1,-0.2,-0.9,-0.5,-0.3,-0.5,-0.2,-0.5,-0.2,23.33,Standard
3,4,-1.6,0.2,0.1,-0.3,-0.7,-0.4,-0.1,0.0,-0.5,0.1,41.67,Standard
4,5,-1.6,-0.3,-0.2,-0.1,-0.3,0.2,-0.3,-0.2,-0.3,-0.1,41.67,Standard
5,6,8.4,1.7,0.8,0.9,1.0,0.9,0.7,0.8,1.0,0.6,100.00,Exceptional
6,7,-8.2,-1.6,-1.0,-1.3,-1.0,-0.6,-0.1,-1.0,-1.2,-0.4,13.33,Weak
7,8,2.6,-0.1,0.3,0.4,0.5,0.4,0.2,0.3,0.5,0.1,66.67,Strong
8,9,4.8,0.2,0.6,0.4,0.7,0.6,0.5,0.6,0.8,0.4,83.33,Exceptional
9,10,6.8,1.3,0.6,0.9,0.8,0.7,0.5,0.6,1.0,0.4,93.33,Exceptional


In [16]:
final = macro.merge(scored, on='TeamNumber').merge(roster, on='TeamNumber')

In [17]:
final

,Unnamed: 0,FullName,Email,TeamNumber,TeammateNumber,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,...,workAllocationDiff,teamRoleDiff,enjoyDiff,conflictResDiff,efficiencyDiff,expressionDiff,percentile,percentileCategory,index,Group
0,0,Arush Godha,arush.godha@berkeley.edu,1,1,6.0,6.5,6.3,0.9,6.0,...,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong,0,101-1
1,1,Aaron Lee,aaronlee5702@berkeley.edu,1,2,6.0,6.5,6.3,0.9,6.0,...,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong,0,101-1
2,2,Naomi Nogueira,naomi_nogueira@berkeley.edu,1,3,7.0,6.5,6.3,0.9,7.0,...,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong,0,101-1
3,3,Khant Nyi,knyi@berkeley.edu,1,4,6.0,6.5,6.3,0.9,4.0,...,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong,0,101-1
4,4,Brandon Ramos-Bailon,bramos6837@berkeley.edu,1,5,7.0,6.5,6.3,0.9,7.0,...,0.3,0.4,0.4,0.1,0.3,0.3,70.00,Strong,0,101-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,169,Pablo Brunet,pbrunet@berkeley.edu,30,2,6.0,6.3,6.3,0.9,6.0,...,0.3,0.2,0.0,0.1,0.3,-0.1,53.33,Strong,168,105-6
170,170,Eilyn Garcia,garciaeilyn@berkeley.edu,30,3,7.0,6.3,6.3,0.9,7.0,...,0.3,0.2,0.0,0.1,0.3,-0.1,53.33,Strong,168,105-6
171,171,Tiffelyn Kurniawan,tifkurnia2004@berkeley.edu,30,4,NaN,6.3,6.3,0.9,NaN,...,0.3,0.2,0.0,0.1,0.3,-0.1,53.33,Strong,168,105-6
172,172,Eric Lind,elind1@berkeley.edu,30,5,NaN,6.3,6.3,0.9,NaN,...,0.3,0.2,0.0,0.1,0.3,-0.1,53.33,Strong,168,105-6


In [18]:
final.to_csv("E29_CHECKIN01_FULL_ANALYSIS_V1.csv")